In [139]:
from urllib.parse import urlparse
from re import compile
import ipaddress
import re
import whois
from datetime import datetime,timezone
from bs4 import BeautifulSoup
import urllib
import urllib.request
import lxml
import requests
from string import punctuation

In [13]:
url_string='https://www.geeksforgeeks.org/ternary-operator-in-python/'

In [14]:
class UrlFeatures(object):
    def __init__(self,url):
        self.description = 'URL'
        self.url = url
        self.urlparse = urlparse(self.url)
        self.host = self.urlparse.hostname
        self.today = datetime.now().replace(tzinfo=None)
        try:
            self.whois=whois.whois(self.urlparse.netloc)
        except:
            self.whois=None
class LexicalFeatures(UrlFeatures):
    def url_length(self):
        return len(self.url)
    
    def url_scheme(self):
        return self.urlparse.scheme
    def url_path_length(self):
        return len(self.urlparse.path)
    def url_host_length(self):
        return len(self.host)
    def url_ip_address(self):
        try:
            ipaddress.ip_address(self.urlparse.netloc)
            ip=1
        except ValueError:
            ip=0
        return ip
    def url_number_of_digits(self):
        digits = [i for i in self.url if i.isdigit()]
        return len(digits)
    def url_number_of_parameters(self):
        params = self.urlparse.query
        return 0 if params == '' else len(params.split('&'))
    def url_number_of_fragments(self):
        frags = self.urlparse.fragment
        return len(frags.split('#')) if frags!='' else 0

    def is_url_encoded(self):
        return '%' in self.url.lower()

    def url_number_encoded_char(self):
        encs = [i for i in self.url if i == '%']
        return len(encs)

#     def url_string_entropy(self):
#         return self.__get_entropy(self.url)

    def url_number_of_subdirectories(self):
        d = self.urlparse.path.split('/')
        return len(d)-1

    def url_number_of_periods(self):
        periods = [i for i in self.url if i == '.']
        return len(periods)

    def url_has_client_in_string(self):
        return 'client' in self.url.lower()

    def url_has_admin_in_string(self):
        return 'admin' in self.url.lower()

    def url_has_server_in_string(self):
        return 'server' in self.url.lower()

    def url_has_login_in_string(self):
        return 'login' in self.url.lower()
        
    def url_get_tld(self):
          return self.urlparse.netloc.split('.')[-1].split(':')[0]
    def url_number_of_subdomains(self):
        subdomains = len(self.urlparse.netloc.split(':')[0].split('.'))-1
        return subdomains if subdomains>=0 else 0 
    def url_number_of_dashes(self):
        return 1 if len([i for i in self.urlparse.netloc if i=='-'])>0 else 0 
    def redirection(self):
        pos = self.url.rfind('//')
        if pos > 6:
            if pos > 7:
                return 1
            else:
                return 0
        else:
            return 0
    def url_have_at_sign(self):
          return 1 if '@' in self.url else 0
    def is_tiny_url(self):
        #listing shortening services
            shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"
            match=re.search(shortening_services,self.url)
            if match:
                return 1
            else:
                return 0
    def daysSinceRegistration(self):
        if self.whois and self.whois['creation_date']:
            diff = self.today - self.whois['creation_date'].replace(tzinfo=None)
            diff = str(diff).split(' days')[0]
            return diff
        else:
            return 0

    def daysSinceExpiration(self):
        if self.whois and self.whois['expiration_date']:
            diff = self.whois['expiration_date'].replace(tzinfo=None) - self.today
            diff = str(diff).split(' days')[0]
            return diff
        else:
            return 0
        
    def url_number_of_uppercase(self):
        return len([i for i in self.url if i.isupper()])
    
    def url_number_of_spaces(self):
        return self.url.count("%20")
    def url_number_of_zeroes(self):
        return self.url.count('0')
    def is_url_single_character_directory(self):
        for i in self.urlparse.path.split('/'):
            if len(i)==1:
                return 1
        return 0
    def url_ratio_upper_to_lower(self):
        l2=len([i for i in self.url if i.islower()])
        return len([i for i in self.url if i.isupper()])/(l2 if l2>0 else 1000000)
    def is_url_in_dns_record(self):
        return 1 if not self.whois.registrar else 0

    def url_web_traffic(self):
        try:
            url = urllib.parse.quote(self.url)
            rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
                "REACH")['RANK']
            rank = int(rank)
            return rank
        except TypeError:
                return 0
    

163

In [166]:
class ContentFeatures(UrlFeatures):
    def __init__(self,url):
        try:
            self.response = requests.get(url)
        except:
            self.response=None
        if self.response:
            self.content = BeautifulSoup(self.response.text, "html.parser")
            self.html = self.content.find('html')
            self.scripts=""
            for i in self.content.findAll('script'):
                self.scripts+=i.text
        else:
            self.content = ""
    def is_iframe_border(self):
        if self.content:
            for i in c.content.findAll("iframe"):
                if i.get('frameborder'):
                    return i['frameborder']
            return 0
        else:
            return 0
    def is_status_bar_mouseover_url(self):
        if not self.content:
            return 0
        else:
            for i in c.content.findAll('a'):
                if i.get('onmouseover'):
                    return 1
            return 0
    def is_right_click(self):
          if self.content == "":
            return 0
          else:
            if re.findall(r"event.button ?== ?2", str(self.content)) or str(self.content).find('''document.addEventListener('contextmenu',''')>0 or str(self.content).find('contextmenu')>0:
                return 1
            else:
                return 0
    def is_forwarding(self):
        if self.content == "":
            return 0
        else:
            count=0
            for i in self.content.findAll('script'):
                count+=i.text.count('history')
            return count
    def number_of_embeds(self):
        return len(self.content.findAll('embed')) if self.content else 0
    def number_of_embeds(self):
        return len(self.content.findAll('a')) if self.content else 0
    def number_of_objects(self):
        return len(self.content.findAll('object')) if self.content else 0
    def number_pf_scripts(self):
        return len(self.content.findAll('script')) if self.content else 0
    def script_to_body_ratio(self):
        scriptText=0
        for i in self.content.findAll('script'):
            scriptText+=len(i.text)
        htmlText=len(self.content.find('html').text)
        return scriptText/htmlText
    def html_length(self):
        return len(self.content.find('html').text) if self.content else 0
    
    def number_of_page_tokens(self):
        html_tokens = len(self.html.text.lower().split()) if self.html else None
        return html_tokens

    def number_of_sentences(self):
        html_sentences = len(self.html.text.split('.')) if self.html else None
        return html_sentences

    def number_of_punctuations(self):
        excepts = ['<', '>', '/']
        matches = [i for i in self.html.text if i in punctuation and i not in excepts]
        return len(matches)

    def number_of_distinct_tokens(self):
        html_tokens = [i.strip() for i in self.html.text.lower().split()]
        return len(set(html_tokens))

    def number_of_capitalizations(self):
        uppercases = [i for i in self.html.text if i.isupper()]
        return len(uppercases)
    def suspicious_escape(self):
        return self.scripts.count('escape')
    def suspicious_eval(self):
        return self.scripts.count('eval')
    def suspicious_link(self):
        return self.scripts.count('link')
    def suspicious_unescape(self):
        return self.scripts.count('unescape')
    def suspicious_exec(self):
        return self.scripts.count('exec')
    def suspicious_search(self):
        return self.scripts.count('search')
    def count_of_suspicious_js_function(self):
        return self.suspicious_search()+self.suspicious_exec()+self.suspicious_unescape()+self.suspicious_link()+self.suspicious_eval()+self.suspicious_escape()
    def average_script_length(self):
        return len(self.scripts)/len(self.content.findAll('script'))
c = ContentFeatures('https://developer.mozilla.org/en-US/docs/Web/HTML/Element/iframe')    

In [173]:
c.is_iframe_border()
c.is_status_bar_mouseover_url()
c.is_right_click()
c.is_forwarding()
c.number_pf_scripts()
c.script_to_body_ratio()
c.html_length()
c.number_of_page_tokens()
c.number_of_sentences()
c.number_of_punctuations()
c.number_of_distinct_tokens()
c.number_of_capitalizations()
c.suspicious_escape()
c.suspicious_eval()
c.suspicious_link()
c.suspicious_unescape()
c.suspicious_exec()
c.suspicious_search()
c.count_of_suspicious_js_function()
c.average_script_length()

13694

In [15]:
l = LexicalFeatures(url_string);
l.url_scheme()
l.url_ip_address()
l.url_number_of_digits()
l.url_number_of_parameters()
l.url_number_of_fragments()
l.is_url_encoded()
l.url_number_encoded_char()
l.url_number_of_subdirectories()
l.url_number_of_periods()
l.url_has_client_in_string()
l.url_has_server_in_string()
l.url_has_server_in_string()
l.url_has_login_in_string()
l.url_get_tld()
l.url_number_of_subdomains()
l.url_number_of_dashes()
l.redirection()
l.url_have_at_sign()
l.is_tiny_url()
l.daysSinceRegistration()
l.daysSinceExpiration()
l.url_number_of_uppercase()
l.url_number_of_spaces()
l.url_number_of_zeroes()
l.is_url_single_character_directory()
l.url_ratio_upper_to_lower()
l.is_url_in_dns_record()
l.url_web_traffic()

163

In [20]:
rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url_string).read(), "xml").find(
                "REACH")['RANK']
rank

'163'

In [147]:
'<script>onmouseover(){}</script>'.find('onmouseover')
c.content.findAll('script')

[<script>Array.prototype.flat&&Array.prototype.includes||document.write('<script src="https://polyfill.io/v3/polyfill.min.js?features=Array.prototype.flat%2Ces6"><\/script>')</script>,
 <script defer="defer" src="/static/js/main.2b2344c8.js"></script>,
 <script defer="" src="/static/js/ga.js"></script>,
 <script>const c={light:"#ffffff",dark:"#1b1b1b"};if(window&&document.documentElement){const e=window.localStorage.getItem("theme");e&&(document.documentElement.className=e,document.documentElement.style.backgroundColor=c[e])}</script>,
 <script id="hydration" type="application/json">{"doc":{"isMarkdown":true,"isTranslated":false,"isActive":true,"flaws":{},"title":"<iframe>: The Inline Frame element","mdn_url":"/en-US/docs/Web/HTML/Element/iframe","locale":"en-US","native":"English (US)","sidebarHTML":"\n  <ol>\n   \n            <li><a href=\"/en-US/docs/Web/HTML/Element/audio\"><code>&lt;audio&gt;</code></a></li>\n  \n            <li><a href=\"/en-US/docs/Web/HTML/Element/embed\"><code